In [ ]:
!pip install tensorflow-io
!pip install kafka-python
!pip install sklearn
!pip install pandas
!pip install tensorflow
!pip install matplotlib

In [ ]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaConsumer
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

#Importing the Libraries
import pandas as PD
import numpy as np
%matplotlib inline
import matplotlib. pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib. dates as mandates
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

In [1]:
from kafka import KafkaConsumer

train = []
label = []

consumer = KafkaConsumer(
    'stock',
    bootstrap_servers=['broker:29092'],
    auto_offset_reset='largest',
    enable_auto_commit=False,
    group_id='my-group-id',
    key_deserializer=lambda x: x.decode('utf-8'),
    value_deserializer=lambda x: x.decode('utf-8'),
    consumer_timeout_ms = 10000
)


for i,event in enumerate(consumer):
    event_data = event.value
    event_key = event.key
    # Do whatever you want
    print("Iteration {}: {}-{}".format(i, event_key, event_data))
    train.append(event_key)
    label.append(event.value[3])
    #sleep(0.05)

Iteration 0: 0.02685451599034297,0.02182592494011179,0.02311808055994885,0.0014455920330678264-2000,1,27,3.97702791979
Iteration 1: 0.02893805602407648,0.02182592494011179,0.02311808055994885,0.009588483864905581-2000,1,28,4.07350291966
Iteration 2: 0.02893805602407648,0.02182592494011179,0.027324427095601844,0.04053147282588905-2000,1,31,4.0411735314
Iteration 3: 0.02893805602407648,0.02182592494011179,0.027324427095601844,0.004355074732533451-2000,2,1,4.08941103133
Iteration 4: 0.027019876310480556,0.02182592494011179,0.02749268095702795,0.01302862693094041-2000,2,2,4.07452924944
Iteration 5: 0.02685451599034297,0.02182592494011179,0.027324427095601844,0.006605806632120067-2000,2,3,4.10531914301
Iteration 6: 0.029004200152131537,0.01982965131754058,0.02749268095702795,0.006715598432099902-2000,2,4,4.04373935587
Iteration 7: 0.02685451599034297,0.02182592494011179,0.027324427095601844,0.0016651756330274962-2000,2,7,4.10531914301
Iteration 8: 0.02685451599034297,0.02182592494011179,0.0

In [ ]:
print("tensorflow version: {}".format(tf.__version__))

In [ ]:
train = list(map(lambda x: x.split(","), train))

In [ ]:
label = list(map(lambda x: float(x), label))

Get message from Kafka

In [ ]:
for temp in train:
    for j in range(len(temp)):
        temp[j] = float(temp[j])
# z = []
# for i in range(len(train)):
#     z.append(train[i].pop(4))

In [ ]:
train = list(map(lambda x: tf.convert_to_tensor(x), train))
output_var = list(map(lambda x: tf.convert_to_tensor(x), label))
for x in output_var:
    tf.reshape(x, (1,1), name=None)
for x in train:
    tf.reshape(x, (1,4), name=None)    

In [ ]:
#Splitting to Training set and Test set
timesplit= TimeSeriesSplit(n_splits=10)
for train_index, test_index in timesplit.split(train):
        X_train_list, X_test_list = train[:len(train_index)], train[len(train_index): (len(train_index)+len(test_index))]
        y_train_list, y_test_list = output_var[:len(train_index)], output_var[len(train_index): (len(train_index)+len(test_index))]     

In [ ]:
X_train = tf.convert_to_tensor(X_train_list, dtype=None, dtype_hint=None, name=None)
y_train = tf.convert_to_tensor(y_train_list, dtype=None, dtype_hint=None, name=None)
X_test = tf.convert_to_tensor(X_test_list, dtype=None, dtype_hint=None, name=None)
y_test = tf.convert_to_tensor(y_test_list, dtype=None, dtype_hint=None, name=None)

In [ ]:
X_train  = tf.reshape(X_train, (len(X_train), 1, 4))
X_test = tf.reshape(X_test, (len(X_test), 1, 4))
y_train  = tf.reshape(y_train, (len(y_train), 1, 1))
y_test  = tf.reshape(y_test, (len(y_test), 1, 1))

In [ ]:
# Set CPU as available physical device
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')

# To find out which devices your operations and tensors are assigned to
tf.debugging.set_log_device_placement(True)

In [ ]:
#Building the LSTM Model
# Có thể thử nghiệm thêm về các mô hình mạng khác
lstm = Sequential()
# Chú ý về input shape cần thay đổi
lstm.add(LSTM(1, input_shape=(1, 4), activation='relu', return_sequences=False))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
# plot_model(lstm, show_shapes=True, show_layer_names=True)

In [ ]:
# Cần thay đổi batch_size để học tốt hơn, batch size càng nhỏ học càng tốt và kết quả sẽ tốt hơn, nhỏ quá sẽ bị overfit
history=lstm.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1, shuffle=False)

In [ ]:
#LSTM Prediction
y_pred= lstm.predict(X_test)

In [ ]:
pred = []
for y in y_pred:
  pred.append(y[0])
  
true = []
for y in y_test:
  true.append(y[0][0])

In [ ]:
#Predicted vs True Adj Close Value – LSTM
plt.plot(list(true), label='True Value')
plt.plot(list(pred), label='LSTM Value')
plt.title('Prediction by LSTM')
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt.legend()
plt.show()